In [85]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from os import path
import json
import pandas as pd
import numpy as np
import scipy as sp
import pickle
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline

In [86]:
FILE_NEXT_TO_GO = 'next_to_go.pkl'
with open(FILE_NEXT_TO_GO, 'rb') as f:
    meeting = pickle.load(f)
print(meeting.keys())
print('distance {}'.format(meeting['raceDistance']))
horses = meeting['runners']

dict_keys(['raceNumber', 'raceName', 'raceDistance', 'trackDirection', 'meeting', 'skyRacing', 'hasParimutuel', 'hasFixedOdds', 'broadcastChannel', 'broadcastChannels', 'hasForm', 'hasEarlySpeedRatings', 'allIn', 'cashOutEligibility', 'allowBundle', 'willHaveFixedOdds', 'fixedOddsOnlineBetting', 'raceStartTime', 'raceClassConditions', 'apprenticesCanClaim', 'prizeMoney', 'raceStatus', 'substitute', 'results', 'pools', 'allowMulti', 'allowParimutuelPlace', 'parimutuelPlaceStatus', 'allowFixedOddsPlace', 'numberOfPlaces', 'numberOfFixedOddsPlaces', 'runners', 'oddsUpdateTime', 'fixedOddsUpdateTime', 'tips', 'ratings', 'multiLegApproximates', 'betTypes'])
distance 1600


In [88]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
dfo = pd.read_sql_table('race', conn, columns=[
    'runner_name', 'distance', 'handicap', 'time'])
dfd = dfo[dfo.distance == meeting['raceDistance']]
dfd.head()

,runner_name,distance,handicap,time
66,LUCQUIDE,1600,65.5,113.5
69,LUCQUIDE,1600,55.5,101.7
70,LUCQUIDE,1600,58.5,107.6
71,MATOVANI,1600,58.5,113.3
91,GERONIMO,1600,58.5,99.4


In [90]:
df = dfd
df = df[(df.time > 0)]
df = df[(df.handicap > 0)]
del df['distance']
df.head()

,runner_name,handicap,time
66,LUCQUIDE,65.5,113.5
69,LUCQUIDE,55.5,101.7
70,LUCQUIDE,58.5,107.6
71,MATOVANI,58.5,113.3
91,GERONIMO,58.5,99.4


In [ ]:
encoder = LabelEncoder()


In [79]:
X = Z[:,0:2]
print('X')
print(X[:5])

Y = Z[:,-1]
print('Y')
print(Y[:5])

X
[[  98.    65.5]
 [  98.    55.5]
 [  98.    58.5]
 [ 104.    58.5]
 [  65.    58.5]]
Y
[ 113.5  101.7  107.6  113.3   99.4]


In [80]:
def base_model():
    model = Sequential()
    model.add(Dense(2, input_dim=2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [83]:
estimator = KerasRegressor(build_fn=base_model, nb_epoch=100, batch_size=5, verbose=0)

In [84]:
seed = np.random.seed(123)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: {:.2f} ({:.2f}) MSE".format(results.mean(), results.std()))

Results: 3570.33 (4085.33) MSE
